In [ ]:
def safe_division(number, divisor, /, ndigits=10, *, ignore_overflow=False, ignore_zero_division=False):
    try:
        return round(number / divisor, ndigits)
    except OverflowError:
        if ignore_overflow:
            return 0
        raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        raise

In [ ]:
safe_division(10, 11)

In [ ]:
from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) '
        f'-> {result!r}')
        return result
    return wrapper

@trace
def fib(n):
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

fib(3)

In [ ]:
import pickle
pickle.dumps(fib)
print(fib.__name__)
print(fib.__module__)
print(fib.__annotations__)

help("while")

In [ ]:
import math
class SimpleContainer:
    def __init__(self, *range):
        if not range:
            raise Exception("At length one argument must be provided.")
        if len(range) == 1:
            self._start = 0
            self._end = range[0]
            self._step = 1
        elif len(range) == 2:
            self._start, self._end = range
            self._step = 1
        elif len(range) == 3:
            self._start, self._end, self._step = range
        else:
            raise ValueError("Atmost 3 arguments are expected.")

        # calculate length on the basis of start, end and step
        self.length = math.ceil((self._end - self._start) / self._step)

    def __next__(self):
        if self.start >= self.end:
            raise StopIteration
        else:
            self.start += self.step
            return self.start - self.step

    def __iter__(self):
        # reset the start values 
        self.start = self._start
        self.end = self._end
        self.step = self._step
        
        return self

    def __len__(self):
        return self.length
    
    def __list__(self):
        for i in self:
            yield i


In [ ]:
l = SimpleContainer(5, 15, 3)
for i in l:
    print(i, end=" ")
print()
print("length -> ", len(l))


In [ ]:
import itertools

In [ ]:
res = list(itertools.repeat(SimpleContainer(5, 15, 3), 3))
for r in res:
    for a in r:
        print(a)

In [ ]:
it = itertools.cycle(SimpleContainer(1, 5))
for _ in range(10):
    print(next(it), end=" ")

In [ ]:
combined_iter = itertools.zip_longest(SimpleContainer(1, 5), 
                                      SimpleContainer(3, 30, 6))
for elements in combined_iter:
    print(elements, end=" ")


In [ ]:
values = [1,2,3,5,5,6]
starting_3 = itertools.islice(values, 3)
odds = itertools.islice(values,1, None, 2)
evens = itertools.islice(values, 0, None, 2)
print(list(starting_3))
print(list(odds))
print(list(evens))


In [ ]:
items = [1, 2, 3, 4, 0, -1, -2, 10]
less_than_3 = lambda x: x < 3
it = filter(less_than_3, items)
it2 = itertools.filterfalse(less_than_3, items)

print(list(it))
print(list(it2))

In [ ]:
items = SimpleContainer(1, 20, 4)
sum_reduce = lambda x, y: (x + y) % 20
result = itertools.product(items, sum_reduce)
print(list(result))

In [ ]:
items = SimpleContainer(1, 3)
alpha = ["a", "b", "c", "d"]
single = itertools.product(items, repeat=2)
print(list(single))
multiple = itertools.product(items, alpha)
print(list(multiple))


In [ ]:
it = itertools.permutations([1,2,3], 2)
print(list(it))

In [ ]:
it = itertools.combinations_with_replacement([1,2,3], 2)
print(list(it))

In [ ]:
from collections import defaultdict

In [ ]:
class SimpleGradeBook:
    def __init__(self):
        self._grades = defaultdict(lambda: defaultdict(int))

    #  def add_student(self, name):
    #      self._grades[name] = []
    def report_grade(self, name, subject, score):
        self._grades[name][subject] = score

    def average_grade(self, name):
        print(self._grades)
        print(self._grades[name]["Beniwal"])
        grades = self._grades[name]
        marks = 0
        for subject, grade in grades.items():
            marks += grade
        return marks / len(grades.keys())


book = SimpleGradeBook()
# book.add_student('Isaac Newton')
book.report_grade("Isaac Newton", "English", 90)
book.report_grade("Isaac Newton", "English", 95)
book.report_grade("Isaac Newton", "English", 85)
book.average_grade("Isaac Newton")


In [ ]:
from collections import namedtuple

Grade = namedtuple("Grade", ("score", "weight"))

class Subject:
    def __init__(self):
        self._grades = []

    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
    
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

class Student:
    def __init__(self):
        self._subjects = defaultdict(Subject)
    
    def get_subjects(self, name):
        return self._subjects[name]

    def average_grade(self):
        total, count = 0, 0 
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count
    
class GradeBook:
    def __init__(self) -> None:
        self._students = defaultdict(Student)
    
    def get_student(self, name):
        return self._students[name]



In [ ]:
book = GradeBook()
albert  = book.get_student("Albert Einstein")
math = albert.get_subjects("Math")
english = albert.get_subjects("English")
math.report_grade(80, 0.10)
english.report_grade(90, 0.02)
albert.average_grade()

In [ ]:
from functools import reduce


class C:
    _oncreate = []

    def __new__(cls):
        res = reduce(
            lambda x, y: y(x), 
            cls._oncreate,
            super().__new__(cls)
        )
        c = lambda z : z 
        res = c(res)
        print("res", res)
        return res
    
    @classmethod
    def oncreate(cls, func):
        print("Oncreate func", func)
        cls._oncreate.append(func)
        return func


In [ ]:
@C.oncreate
def spew(obj):
    print("Called", obj)
    obj.spew = 'c'
    return obj

@C.oncreate
def view(obj):
    print("Called", obj)
    obj.view = 'view'
    return obj

In [ ]:

d = C()
d.view
# e = C()
# f = C()
# spew(c)
# print(c._oncreate)
# print(C._oncreate)
# spew(d)
# print(c._oncreate)
# print(C._oncreate)

In [ ]:
d(lambda x : x )

In [93]:
class delegate:
    def __init__(self, func):
        self.callbacks = []
        self.basefunc = func
        print("Here Here")

    def __iadd__(self, func):
        if callable(func):
            print("Here Here")
            # self.__isub__(func)
            self.callbacks.append(func)
        return self

    def callback(self, func):
        if callable(func):
            print("callable called")
            # self.__isub__(func)
            self.callbacks.append(func)
        return func

    # def __isub__(self, func):
    #     try:
    #         print("isub", func)
    #         self.callbacks.remove(func)
    #     except ValueError:
    #         pass
    #     return self

    def __call__(self, *args, **kwargs):
        result = self.basefunc(*args, **kwargs)
        for func in self.callbacks:
            newresult = func(result)
            result = result if newresult is None else newresult
        print("result", result)
        return result


In [94]:
@delegate
def intfactory(num):
    return int(num)



def increment(num):
    return num + 1

intfactory += increment
intfactory += lambda x : x * 2 
intfactory += lambda x : x * 3
@intfactory.callback
def notify(num):
    print("Number is", num)

print(intfactory(3))


Here Here
Here Here
Here Here
Here Here
callable called
Number is 24
result 24
24


In [70]:
class a:
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args, **kwargs):
        print("Called")
        return self.func(*args, **kwargs)

In [71]:
@a
def fn():
    print("fn")